# STA 141B Assignment 3

Due __Feb 11, 2019__ by 11:59pm. Submit by editing this file, committing the changes with git, and then pushing to your private GitHub repo for the assignment. This assignment will be graded according to the class rubric.

Please do not rename this file or delete the exercise cells, because it will interfere with our grading tools. Put your answers in new cells after each exercise. You can make as many new cells as you like. Use code cells for code and Markdown cells for text. Answer all questions with complete sentences.

The purpose of this assignment is to practice using web APIs and combining data from multiple sources.

## USDA Food Composition

The US Department of Agriculture publishes price estimates for fruits and vegetables [online](https://www.ers.usda.gov/data-products/fruit-and-vegetable-prices/fruit-and-vegetable-prices/). The most recent estimates are based on a 2013 survey of US retail stores. We've cleaned and merged the fruit and vegetable price estimates for you. The result is the `fresh.csv` file included in the repository.

In this assignment, you'll combine the price estimates with nutrition data from the [USDA Food Composition Databases](https://ndb.nal.usda.gov/ndb/search/list), and then analyze the resulting dataset.

The USDA Food Composition Databases have a [documented](https://ndb.nal.usda.gov/ndb/doc/index) web API that returns data in JSON format . You need a key in order to use the API. Only 1000 requests are allowed per hour, so it's a good idea to use caching.

[Sign up for an API key here](https://api.data.gov/signup/). The key will work with any data.gov API. You may need the key again later in the quarter, so make sure you save it.

__Exercise 1.1__

Read the [search request documentation](https://ndb.nal.usda.gov/ndb/doc/apilist/API-SEARCH.md), then write a function called `ndb_search()` that makes a search request. The function should accept the search term as an argument. The function should return the search result items as a list (for 0 items, return an empty list).

Note that the search url is: `https://api.nal.usda.gov/ndb/search`

As an example, a search for `"quail eggs"` might return this list:

```python
[{'offset': 0,
  'group': 'Branded Food Products Database',
  'name': 'EL COMPI, QUAIL EGGS, UPC: 854955002226',
  'ndbno': '45362205',
  'ds': 'LI',
  'manu': "milly's desserts llc"},
 {'offset': 1,
  'group': 'Branded Food Products Database',
  'name': 'BUDDHA, QUAIL EGGS IN BRINE, UPC: 761934535098',
  'ndbno': '45099560',
  'ds': 'LI',
  'manu': 'Sung Ly International Corporation'},
 {'offset': 2,
  'group': 'Branded Food Products Database',
  'name': 'GRAN SABANA, QUAIL EGGS, UPC: 819140010103',
  'ndbno': '45169279',
  'ds': 'LI',
  'manu': 'L & M C Farms, Inc.'},
 {'offset': 3,
  'group': 'Branded Food Products Database',
  'name': 'L&W, QUAIL EGGS, UPC: 024072000256',
  'ndbno': '45094890',
  'ds': 'LI',
  'manu': 'L&W International Co.'},
 {'offset': 4,
  'group': 'Branded Food Products Database',
  'name': 'CHAOKOH, QUAIL EGG IN BRINE, UPC: 044738074186',
  'ndbno': '45094707',
  'ds': 'LI',
  'manu': 'Theppadung Porn Coconut Co'},
 {'offset': 5,
  'group': 'Dairy and Egg Products',
  'name': 'Egg, quail, whole, fresh, raw',
  'ndbno': '01140',
  'ds': 'SR',
  'manu': 'none'}]
```

As usual, make sure you document and test your function.

In [29]:
import requests
import requests_cache
import pandas as pd
import time

requests_cache.install_cache("mycache")

In [30]:
def read_key(keyfile):
    with open(keyfile) as f:
        return f.readline().strip("\n")

In [31]:
key = read_key("C:/Users/dthed/Downloads/Keys/data_gov.txt")

In [197]:
def ndb_search(query):
    response = requests.get("https://api.nal.usda.gov/ndb/search", params = {
        "api_key": key,
        "q": query
    })

    response.raise_for_status()
    response = response.json()
    if "errors" in response:
        return []
    else:
        return response["list"]["item"]

In [198]:
ndb_search("quail eggs")

[{'offset': 0,
  'group': 'Branded Food Products Database',
  'name': 'EL COMPI, QUAIL EGGS, UPC: 854955002226',
  'ndbno': '45362205',
  'ds': 'LI',
  'manu': "milly's desserts llc"},
 {'offset': 1,
  'group': 'Branded Food Products Database',
  'name': 'BUDDHA, QUAIL EGGS IN BRINE, UPC: 761934535098',
  'ndbno': '45099560',
  'ds': 'LI',
  'manu': 'Sung Ly International Corporation'},
 {'offset': 2,
  'group': 'Branded Food Products Database',
  'name': 'GRAN SABANA, QUAIL EGGS, UPC: 819140010103',
  'ndbno': '45169279',
  'ds': 'LI',
  'manu': 'L & M C Farms, Inc.'},
 {'offset': 3,
  'group': 'Branded Food Products Database',
  'name': 'L&W, QUAIL EGGS, UPC: 024072000256',
  'ndbno': '45094890',
  'ds': 'LI',
  'manu': 'L&W International Co.'},
 {'offset': 4,
  'group': 'Branded Food Products Database',
  'name': 'CHAOKOH, QUAIL EGG IN BRINE, UPC: 044738074186',
  'ndbno': '45094707',
  'ds': 'LI',
  'manu': 'Theppadung Porn Coconut Co'},
 {'offset': 5,
  'group': 'Dairy and Egg Pro

In [199]:
ndb_search("fdsklfds")

[]

__Exercise 1.2__

Use your search function to get NDB numbers for the foods in the `fresh.csv` file. It's okay if you don't get an NDB number for every food, but try to come up with a strategy that gets most of them. Discuss your strategy in a short paragraph.

Hints:

* The foods are all raw and unbranded.
* You can test search terms with the [online search page](https://ndb.nal.usda.gov/ndb/search/list).
* The [Python](https://docs.python.org/3/library/stdtypes.html#string-methods) and [Pandas](http://pandas.pydata.org/pandas-docs/stable/user_guide/text.html) string methods are useful here. It's okay if you use _simple_ regular expressions in the Pandas methods, although this exercise can be solved without them.
* You can merge data frames that have a column in common with `pd.merge()`.

In [35]:
fresh_complete = pd.read_csv("fresh.csv")

#fresh_complete.head()

In [143]:
fresh_complete

,form,price_per_lb,yield,lb_per_cup,price_per_cup,food,type
0,Fresh1,0.316639,0.520000,0.330693,0.201366,watermelon,fruit
1,Fresh1,0.520794,0.510000,0.374786,0.382718,cantaloupe,fruit
2,Fresh1,1.480396,0.740000,0.407855,0.815929,tangerines,fruit
3,Fresh1,2.508450,0.940000,0.319670,0.853060,strawberries,fruit
4,Fresh1,1.987999,0.940000,0.363763,0.769319,plums,fruit
5,Fresh1,1.097122,0.680000,0.407855,0.658039,oranges,fruit
6,Fresh1,6.877702,0.960000,0.319670,2.290205,raspberries,fruit
7,Fresh1,2.078430,0.560000,0.341717,1.268275,pomegranate,fruit
8,Fresh1,0.652795,0.510000,0.363763,0.465612,pineapple,fruit
9,Fresh1,3.087138,0.930000,0.363763,1.207511,apricots,fruit


In [200]:
def narrow_search(food, form, kind, source = 'SR', manu = 'none', exclude = ''):
    
    search_term = "raw " + food + " " + form.lstrip("Fresh,").replace("consumed", "").replace("cabbage", "").strip()
    
    result = ndb_search(search_term)
    
    if not result and kind == 'fruit':
        result = ndb_search("raw " + food + kind)
    
    result_df = pd.DataFrame(result)
    
    crit1 = result_df["ds"] == source
    crit2 = result_df["manu"] == manu
    crit3 = result_df["group"] == "Fruits and Fruit Juices"
    
    if kind == "fruit":
        all_crit = crit1 & crit2 & crit3
    else:
        crit4 = result_df["group"] == "Vegetables and Vegetable Products"
        all_crit = crit1 & crit2 & (crit3 | crit4)
    
    if exclude:
        crit5 = ~result_df["name"].str.contains(exclude, case = False)
        all_crit = all_crit & crit5
    
    result_df = result_df[all_crit]
    
    return result_df

In [195]:
fresh = fresh_complete[["form", "food", "type"]]

for x in range(len(fresh)):
    row = fresh.iloc[x,:]
    food = row["food"].replace("_", " ")
    form = row["form"].rstrip('1')
    kind = row["type"]

    results = narrow_search(food, form, kind, exclude = "juice|-|acerola|muscadine|scallop")
    
    if results.empty:
        print("*******NO RESULTS*******")
    else:
        print(results.iloc[0,:][["name", "ndbno"]])
        
    print(" ")

    time.sleep(0.1)

raw watermelon 
name     Watermelon, raw
ndbno              09326
Name: 2, dtype: object
 
raw cantaloupe 
name     Melons, cantaloupe, raw
ndbno                      09181
Name: 0, dtype: object
 
raw tangerines 
name     Tangerines, (mandarin oranges), raw
ndbno                                  09218
Name: 1, dtype: object
 
raw strawberries 
name     Strawberries, raw
ndbno                09316
Name: 0, dtype: object
 
raw plums 
name     Plums, raw
ndbno         09279
Name: 0, dtype: object
 
raw oranges 
name     Oranges, raw, California, valencias
ndbno                                  09201
Name: 3, dtype: object
 
raw raspberries 
name     Raspberries, raw
ndbno               09302
Name: 0, dtype: object
 
raw pomegranate 
name     Pomegranates, raw
ndbno                09286
Name: 1, dtype: object
 
raw pineapple 
name     Pineapple, raw, all varieties
ndbno                            09266
Name: 0, dtype: object
 
raw apricots 
name     Apricots, raw
ndbno            09021
Na

__Exercise 1.3__

Read the [food reports V2 documentation](https://ndb.nal.usda.gov/ndb/doc/apilist/API-FOOD-REPORTV2.md), then write a function called `ndb_report()` that requests a _basic_ food report. The function should accept the NDB number as an argument and return the list of nutrients for the food.

Note that the report url is: `https://api.nal.usda.gov/ndb/V2/reports`

For example, for `"09279"` (raw plums) the first element of the returned list should be:

```python
{'nutrient_id': '255',
 'name': 'Water',
 'derivation': 'NONE',
 'group': 'Proximates',
 'unit': 'g',
 'value': '87.23',
 'measures': [{'label': 'cup, sliced',
   'eqv': 165.0,
   'eunit': 'g',
   'qty': 1.0,
   'value': '143.93'},
  {'label': 'fruit (2-1/8" dia)',
   'eqv': 66.0,
   'eunit': 'g',
   'qty': 1.0,
   'value': '57.57'},
  {'label': 'NLEA serving',
   'eqv': 151.0,
   'eunit': 'g',
   'qty': 1.0,
   'value': '131.72'}]}
```

Be sure to document and test your function.

In [27]:
def ndb_report(ndbno):
    response = requests.get("https://api.nal.usda.gov/ndb/V2/reports", params = {
        "api_key": key,
        "ndbno": [ndbno]
    })

    response.raise_for_status()
    response = response.json()
    
    if response["notfound"] is not 0:
        print("Food not found in database. Please try again.")
        return []
    else:
        nutrients = response["foods"][0]["food"]["nutrients"]
        return nutrients

In [28]:
plums = ndb_report("09279")
plums[0]

{'nutrient_id': '255',
 'name': 'Water',
 'derivation': 'NONE',
 'group': 'Proximates',
 'unit': 'g',
 'value': '87.23',
 'measures': [{'label': 'cup, sliced',
   'eqv': 165.0,
   'eunit': 'g',
   'qty': 1.0,
   'value': '143.93'},
  {'label': 'fruit (2-1/8" dia)',
   'eqv': 66.0,
   'eunit': 'g',
   'qty': 1.0,
   'value': '57.57'},
  {'label': 'NLEA serving',
   'eqv': 151.0,
   'eunit': 'g',
   'qty': 1.0,
   'value': '131.72'}]}

In [26]:
ndb_report(["5849450", "09279"])

Food not found in database. Please try again.


[]

__Exercise 1.4.__

Answer any two of the questions below. For each answer, state the question, write 2-3 paragraphs, and support your conclusions with plots.

 If you like, you can replace one question with a question you came up with on your own.

* Are fruits typically higher in kilocalories, or vegetables?
* How does yield for fibrous foods (high in dietary fiber) compare to other foods?
* Which food provides the widest variety of nutrients?
* Which nutrients are the most expensive?
* Which foods are the best value for the price? "Best value" might mean highest yield per dollar, most kilocalories per dollar, or some other reasonable definition.
* How do the price distributions compare for fruits and vegetables?